In [1]:
import os
import ROOT 
import pandas as pd
from math import pow, sqrt
from ctypes import c_double
from pprint import pprint

Welcome to JupyROOT 6.28/04


In [2]:
ERA = "2017"
CHANNEL = "Skim3Mu"
MEASURE = "HighPT"
histkey = "ZCand/mass"

WORKDIR = "/home/choij/workspace/ChargedHiggsAnalysis"

DATASTREAM = ""
REGION = ""
if CHANNEL == "Skim1E2Mu": 
    DATASTREAM = "MuonEG"
    REGION = "ZGamma1E2Mu"
if CHANNEL == "Skim3Mu": 
    DATASTREAM = "DoubleMuon"
    REGION = "ZGamma3Mu"
assert DATASTREAM in ["MuonEG", "DoubleMuon"]
assert REGION in ["ZGamma1E2Mu", "ZGamma3Mu"]

CONV = ["DYJets_MG", "DYJets10to50_MG", "ZGToLLG"]
DIBOSON = ["WZTo3LNu_amcatnlo","ZZTo4L_powheg"]
TTX     = ["ttWToLNu", "ttZToLLNuNu", "ttHToNonbb"]
RARE    = ["WWW", "WWZ", "WZZ", "ZZZ", "tZq", "TTG", "VBF_HToZZTo4L", "GluGluHToZZTo4L"]
RARE_noskim = ["tHq", "TTTT", "WWG"]
MCList = CONV + DIBOSON + TTX + RARE + RARE_noskim

SYSTs = []
if CHANNEL == "Skim1E2Mu":
    SYSTs = [["NonpromptUp", "NonpromptDown"],
             ["L1PrefireUp", "L1PrefireDown"],
             ["PileupReweightUp", "PileupReweightDown"],
             ["MuonIDSFUp", "MuonIDSFDown"],
             ["ElectronIDSFUp", "ElectronIDSFDown"],
             ["EMuTrigSFUp", "EMuTrigSFDown"],
             ["JetResUp", "JetResDown"],
             ["JetEnUp", "JetEnDown"],
             ["ElectronResUp", "ElectronResDown"],
             ["ElectronEnUp", "ElectronEnDown"],
             ["MuonEnUp", "MuonEnDown"],
             ["HeavyTagUpCorr", "HeavyTagDownCorr"],
             ["HeavyTagUpUnCorr", "HeavyTagDownUnCorr"],
             ["LightTagUpCorr", "LightTagDownCorr"],
             ["LightTagUpUnCorr", "LightTagDownUnCorr"]]
if CHANNEL == "Skim3Mu":
    SYSTs = [["NonpromptUp", "NonpromptDown"],
             ["L1PrefireUp", "L1PrefireDown"],
             ["PileupReweightUp", "PileupReweightDown"],
             ["MuonIDSFUp", "MuonIDSFDown"],
             ["DblMuTrigSFUp", "DblMuTrigSFDown"],
             ["JetResUp", "JetResDown"],
             ["JetEnUp", "JetEnDown"],
             ["ElectronResUp", "ElectronResDown"],
             ["ElectronEnUp", "ElectronEnDown"],
             ["MuonEnUp", "MuonEnDown"],
             ["HeavyTagUpCorr", "HeavyTagDownCorr"],
             ["HeavyTagUpUnCorr", "HeavyTagDownUnCorr"],
             ["LightTagUpCorr", "LightTagDownCorr"],
             ["LightTagUpUnCorr", "LightTagDownUnCorr"]]


In [3]:
# make a table
data = {}

# make index
indexCol = ["sample", "Central", "Stat"]
for syst in SYSTs:
    indexCol.append(syst[0])
    indexCol.append(syst[1])
indexCol.append("Total")

for index in indexCol:
    data[index] = []

In [4]:
def estTotalErr(sample, sampleDict):
    # find the index of the sample
    idx = sampleDict["sample"].index(sample)
    central = sampleDict["Central"][idx]
    totalErr = pow(sampleDict["Stat"][idx], 2)
    for syst in SYSTs:
        # initialized as False
        if not sampleDict[syst[0]][idx]: continue
        systUp = abs(sampleDict[syst[0]][idx] - central)
        systDown = abs(sampleDict[syst[1]][idx] - central)
        totalErr += pow(max(systUp, systDown), 2)
    
    return sqrt(totalErr)

In [5]:
# data
data["sample"].append("data")

f = ROOT.TFile.Open(f"{WORKDIR}/data/MeasConversion/{ERA}/{CHANNEL}__/DATA/MeasConversion_SkimTree_SS2lOR3l_{DATASTREAM}.root")
h = f.Get(f"{REGION}/{MEASURE}/Central/{histkey}"); h.SetDirectory(0)

stat = c_double()
rate = h.IntegralAndError(0, h.GetNbinsX()+1, stat)
data["Central"].append(rate)
data["Stat"].append(stat.value)

for index in indexCol[3:-1]:
    data[index].append(False)
data["Total"].append(estTotalErr("data", data))

f.Close()

In [6]:
# nonprompt
data["sample"].append("nonprompt")

f = ROOT.TFile.Open(f"{WORKDIR}/data/MeasConvMatrix/{ERA}/{CHANNEL}__/DATA/MeasConvMatrix_SkimTree_SS2lOR3l_{DATASTREAM}.root")
h = f.Get(f"{REGION}/{MEASURE}/Central/{histkey}"); h.SetDirectory(0)
h_up = f.Get(f"{REGION}/{MEASURE}/NonpromptUp/{histkey}"); h.SetDirectory(0)
h_down = f.Get(f"{REGION}/{MEASURE}/NonpromptDown/{histkey}"); h.SetDirectory(0)

stat = c_double()
rate = h.IntegralAndError(0, h.GetNbinsX(), stat)
data["Central"].append(rate)
data["Stat"].append(stat.value)

# systematics
#data["NonpromptUp"].append(h_up.Integral())
#data["NonpromptDown"].append(h_down.Integral())
data["NonpromptUp"].append(rate*1.4)
data["NonpromptDown"].append(rate*0.6)

for index in indexCol[5:-1]:
    data[index].append(False)
data["Total"].append(estTotalErr("nonprompt", data))

f.Close()

In [7]:
for mc in MCList:
    fkey = ""
    if mc in RARE_noskim:
        fkey = f"{WORKDIR}/data/MeasConversion/{ERA}/{CHANNEL}__/MeasConversion_{mc}.root"
    else:
        fkey = f"{WORKDIR}/data/MeasConversion/{ERA}/{CHANNEL}__/MeasConversion_SkimTree_SS2lOR3l_{mc}.root"
    
    assert os.path.exists(fkey)
    f = ROOT.TFile.Open(fkey)
    try:
        h = f.Get(f"{REGION}/{MEASURE}/Central/{histkey}"); h.SetDirectory(0)
    except:
        print(mc); continue
    
    data["sample"].append(mc)
    # fill each row
    stat = c_double()
    rate = h.IntegralAndError(0, h.GetNbinsX()+1, stat)
    data["Central"].append(rate)
    data["Stat"].append(stat.value)
    data['NonpromptUp'].append(False)
    data["NonpromptDown"].append(False)
    
    for index in indexCol[5:-1]:
        try:
            h_syst = f.Get(f"{REGION}/{MEASURE}/{index}/{histkey}")
            h_syst.SetDirectory(0)
            data[index].append(h_syst.Integral())
        except:
            data[index].append(0.)
    f.Close()
    
    data["Total"].append(estTotalErr(mc, data))

DYJets10to50_MG
WWZ
TTG


In [8]:
# make dataframe
df = pd.DataFrame(data)
df.set_index("sample", inplace=True)
df = df.transpose()

pprint(df)

sample                  data  nonprompt  DYJets_MG    ZGToLLG  \
Central                107.0  11.450944  37.494653  51.884055   
Stat                10.34408   1.340956   9.401608   2.862101   
NonpromptUp            False  16.031321      False      False   
NonpromptDown          False   6.870566      False      False   
L1PrefireUp            False      False  37.406265  51.733706   
L1PrefireDown          False      False  37.581652  52.028212   
PileupReweightUp       False      False  37.904874  50.836444   
PileupReweightDown     False      False  36.438891  52.559945   
MuonIDSFUp             False      False  37.849016  52.362411   
MuonIDSFDown           False      False  37.142735  51.409062   
DblMuTrigSFUp          False      False  37.528865  51.927516   
DblMuTrigSFDown        False      False  37.460109  51.840207   
JetResUp               False      False  37.466051  52.199637   
JetResDown             False      False  37.486268  52.112017   
JetEnUp                Fa

In [9]:
df

sample,data,nonprompt,DYJets_MG,ZGToLLG,WZTo3LNu_amcatnlo,ZZTo4L_powheg,ttWToLNu,ttZToLLNuNu,ttHToNonbb,WWW,WZZ,ZZZ,tZq,VBF_HToZZTo4L,GluGluHToZZTo4L,tHq,TTTT,WWG
Central,107.0,11.450944,37.494653,51.884055,14.814611,29.547945,0.065933,0.113343,0.042494,0.107282,0.016672,0.001685,0.083662,0.073834,0.588641,0.064806,0.000114,0.001834
Stat,10.34408,1.340956,9.401608,2.862101,0.867103,0.122477,0.014644,0.018813,0.008789,0.009939,0.002089,0.000332,0.019116,0.002429,0.016878,0.008499,0.000193,0.000821
NonpromptUp,False,16.031321,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
NonpromptDown,False,6.870566,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
L1PrefireUp,False,False,37.406265,51.733706,14.734345,29.463014,0.065459,0.111511,0.041928,0.107009,0.016492,0.001677,0.082974,0.072925,0.586172,0.064328,0.000114,0.001829
L1PrefireDown,False,False,37.581652,52.028212,14.895897,29.630248,0.066393,0.115176,0.043057,0.107551,0.01685,0.001693,0.084392,0.074757,0.591054,0.06528,0.000113,0.001839
PileupReweightUp,False,False,37.904874,50.836444,14.502675,29.222143,0.065127,0.113782,0.039228,0.103892,0.016875,0.0017,0.078726,0.073556,0.580401,0.067627,0.000289,0.00176
PileupReweightDown,False,False,36.438891,52.559945,15.037873,29.851144,0.065212,0.112833,0.045096,0.109813,0.016752,0.001744,0.091848,0.073786,0.595396,0.063707,0.00001,0.001929
MuonIDSFUp,False,False,37.849016,52.362411,14.939381,29.824855,0.066433,0.114241,0.042838,0.108131,0.016799,0.001702,0.084216,0.074404,0.593656,0.065318,0.000114,0.001849
MuonIDSFDown,False,False,37.142735,51.409062,14.690677,29.273042,0.065436,0.11245,0.042152,0.106437,0.016545,0.001668,0.08311,0.073269,0.583661,0.064296,0.000113,0.001819


In [10]:
# measure scale factor
data_convsf = {}

# central value
rate_data = df.loc["Central", "data"]
rate_conv = df.loc["Central", "DYJets_MG"] if MEASURE == "LowPT" else df.loc["Central", "ZGToLLG"]
rate_pred = df.loc["Central", "nonprompt"]
for mc in MCList:
    if mc in CONV: continue
    if not mc in df.columns: continue
    print(mc, df.loc["Central", mc])
    rate_pred += df.loc["Central", mc]
convsf = (rate_data - rate_pred) / rate_conv
data_convsf["Central"] = convsf

WZTo3LNu_amcatnlo 14.814610736183495
ZZTo4L_powheg 29.54794486577193
ttWToLNu 0.06593279096458218
ttZToLLNuNu 0.11334265097742545
ttHToNonbb 0.04249389211762386
WWW 0.10728168090060197
WZZ 0.016671515299004357
ZZZ 0.0016847896730337246
tZq 0.08366175151325903
VBF_HToZZTo4L 0.07383441994902974
GluGluHToZZTo4L 0.5886414271619732
tHq 0.06480567724097588
TTTT 0.00011357564656497739
WWG 0.0018341120301738336


In [11]:
thisConv = "DYJets_MG" if MEASURE == "LowPT" else "ZGToLLG"

# stat
dNconv = df.loc["Stat", thisConv]
dNpred = df.loc["Stat", "nonprompt"]
for mc in MCList:
    if mc in CONV: continue
    if not mc in df.columns: continue
    dNpred += df.loc["Stat", mc]
data_convsf["Stat"] = -(dNpred / rate_conv) + (rate_pred / pow(rate_conv, 2))*dNconv

# systatmatics
for syst in indexCol[3:-1]:
    dNconv = 0.
    if df.loc[syst, thisConv]:
        dNconv += df.loc[syst, thisConv] - df.loc["Central", thisConv]
        
    dNpred = 0.
    if df.loc[syst, 'nonprompt']: dNpred += df.loc[syst, 'nonprompt'] - df.loc['Central', 'nonprompt']
    for mc in MCList:
        if mc in CONV: continue
        if not mc in df.columns: continue
        if df.loc[syst, mc]: dNpred += df.loc[syst, mc] - df.loc['Central', mc]
    data_convsf[syst] = -(dNpred / rate_conv) + (rate_pred / pow(rate_conv, 2))*dNconv

totalErr = pow(data_convsf['Stat'], 2)
for syst in SYSTs:
    thisErr = max( abs(data_convsf[syst[0]]), abs(data_convsf[syst[1]]) )
    totalErr += pow(thisErr, 2)
data_convsf["Total"] = sqrt(totalErr)